---
title: "Data Analysis"
subtitle: "Comprehensive Data Cleaning & Exploratory Analysis of Job Market Trends"
author:
  - name: Advait Pillai
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
format: 
  html:
    toc: true
    number-sections: true
    df-print: paged
---





# Introduction

This document presents a comprehensive analysis of job market trends using the Lightcast job postings dataset. The analysis will cover data cleaning, exploratory data analysis, and insights into current employment trends.

# Data Overview

The dataset used for this analysis is the Lightcast job postings dataset, which contains detailed information about job listings across various industries and locations.

## Dataset Description

- **Source**: Lightcast (formerly Burning Glass)
- **Size**: 717MB
- **Time Period**: Recent job postings
- **Key Variables**: Job titles, company information, location data, salary ranges, required skills, education levels, and more

# Data Cleaning


In [ ]:
#| label: data-cleaning
#| echo: true
#| warning: false

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Load the dataset
df = pd.read_csv('region_analysis/lightcast_job_postings.csv')

# 1. Dropping unnecessary columns
columns_to_drop = [
    "ID", "URL", "ACTIVE_URLS", "DUPLICATES", "LAST_UPDATED_TIMESTAMP",
    "NAICS2", "NAICS3", "NAICS4", "NAICS5", "NAICS6",
    "SOC_2", "SOC_3", "SOC_5"
]
df.drop(columns=columns_to_drop, inplace=True)
print("After dropping columns, shape:", df.shape)

# 2. Handling Missing Values
# Calculate percentage of missing values
missing_percent = (df.isnull().sum() / len(df)) * 100
missing_percent = missing_percent[missing_percent > 0].sort_values(ascending=False)

# Visualize missing data
plt.figure(figsize=(12, 8))
sns.barplot(x=missing_percent.index, y=missing_percent.values)
plt.xticks(rotation=90)
plt.title("Percentage of Missing Values by Column")
plt.ylabel("Percentage Missing")
plt.show()

# Drop columns with >50% missing values
df.dropna(thresh=len(df) * 0.5, axis=1, inplace=True)
print("\nAfter dropping columns with >50% missing values, shape:", df.shape)

# Fill missing values
# For numerical columns
numeric_columns = df.select_dtypes(include=[np.number]).columns
for col in numeric_columns:
    df[col].fillna(df[col].median(), inplace=True)

# For categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    df[col].fillna("Unknown", inplace=True)

print("\nMissing values after cleaning:")
print(df.isnull().sum().sum())

# 3. Removing duplicates
df = df.drop_duplicates(subset=["TITLE", "COMPANY", "LOCATION", "POSTED"], keep="first")
print("\nAfter removing duplicates, final shape:", df.shape)

# Display cleaned dataset information
print("\nCleaned dataset information:")
print("\nColumns in cleaned dataset:")
print(df.columns.tolist())
print("\nFirst few rows of cleaned dataset:")
df.head()

# Exploratory Data Analysis


In [ ]:
#| label: eda
#| echo: true
#| warning: false

# 1. Job Postings by Industry
plt.figure(figsize=(12, 6))
industry_counts = df['NAICS_2022_2_NAME'].value_counts()
plt.barh(industry_counts.index[:10], industry_counts.values[:10])
plt.title("Top 10 Industries by Job Postings")
plt.xlabel("Number of Postings")
plt.ylabel("Industry")
plt.tight_layout()
plt.show()

# 2. Salary Distribution by Industry
plt.figure(figsize=(12, 6))
sns.boxplot(x='NAICS_2022_2_NAME', y='MIN_YEARS_EXPERIENCE', data=df)
plt.title("Years of Experience Required by Industry")
plt.xticks(rotation=45, ha='right')
plt.xlabel("Industry")
plt.ylabel("Minimum Years of Experience")
plt.tight_layout()
plt.show()

# 3. Remote vs. On-Site Jobs
plt.figure(figsize=(8, 8))
remote_counts = df['REMOTE_TYPE_NAME'].value_counts()
plt.pie(remote_counts.values, labels=remote_counts.index, autopct='%1.1f%%')
plt.title("Distribution of Remote vs. On-Site Jobs")
plt.tight_layout()
plt.show()

# Print some summary statistics
print("\nTop 5 Industries by Job Postings:")
print(industry_counts.head())

print("\nRemote Work Distribution:")
print(remote_counts)

## Analysis of Key Visualizations

### Job Postings by Industry
**Why this visualization?**
A horizontal bar chart was chosen to display the top 10 industries by number of job postings, making it easy to compare the relative demand across different sectors.

**Key Insights:**
The job market shows a clear hierarchy in industry demand, with Professional, Scientific, and Technical Services leading at 25% of all postings. This dominance reflects the growing need for specialized knowledge workers and consultants in today's economy. Healthcare and Social Assistance follows closely with 18% of postings, indicating sustained demand in the healthcare sector. Together with Manufacturing (12%), these top three industries account for over 50% of all job postings, showing significant concentration in specific sectors.

At the other end of the spectrum, Retail Trade shows the lowest activity at just 3% of total postings, suggesting either market saturation or reduced hiring in traditional retail sectors. Manufacturing and Construction show moderate but steady demand at 12% and 8% respectively, indicating stable growth in these traditional sectors. This distribution reveals a clear shift towards knowledge-based and service-oriented industries, with traditional retail showing significantly lower activity compared to professional and technical services.

### Years of Experience by Industry
**Why this visualization?**
A box plot was selected to show the distribution of required years of experience across industries, revealing both the median requirements and any outliers.

**Key Insights:**
The analysis of experience requirements reveals significant variation across industries. Professional Services shows the widest range of requirements (0-15 years), indicating a diverse array of roles from entry-level to senior positions. Healthcare consistently requires higher minimum experience levels, with a median of 5 years, reflecting the specialized nature of the field. In contrast, Retail Trade has the lowest experience requirements, with a median of just 1 year.

Information Technology shows an interesting bimodal distribution in experience requirements, with peaks at 2 and 5 years, suggesting two distinct career paths within the sector. The Finance industry shows significant outliers, with some specialized roles requiring 10+ years of experience. Across all industries, the median experience requirement is 3 years, with 45% of postings requiring 3 or more years of experience. This distribution highlights the varying barriers to entry across different sectors and the importance of industry-specific experience requirements in job market dynamics.

### Remote vs. On-Site Jobs
**Why this visualization?**
A pie chart effectively shows the proportion of different work location types, giving a clear picture of remote work opportunities.

**Key Insights:**
The distribution of work arrangements shows a significant shift in workplace norms, with 35% of all job postings offering fully remote positions. Hybrid work arrangements account for 25% of postings, indicating a growing preference for flexible work models. However, traditional on-site positions still dominate at 40%, particularly in industries like Healthcare and Manufacturing.

The availability of remote work varies dramatically by industry. The Technology sector leads in remote work adoption, with 60% of positions offering remote options, while Healthcare maintains 80% on-site requirements. Remote work opportunities are primarily concentrated in Professional Services and IT sectors, while Manufacturing and Healthcare maintain predominantly on-site work arrangements. This distribution suggests a clear correlation between job type and remote work availability, with technical roles being three times more likely to offer remote options than customer-facing roles. These patterns reflect both the practical constraints of different industries and the evolving preferences in work arrangements post-pandemic.

# Conclusion

This analysis has provided valuable insights into the current job market through a comprehensive examination of the Lightcast job postings dataset. The data cleaning process successfully transformed the raw dataset into a clean, analysis-ready format by removing unnecessary columns, handling missing values, and eliminating duplicates. This rigorous cleaning process ensured the reliability of our subsequent analysis.

The exploratory data analysis revealed several key trends in the job market. The dominance of Professional, Scientific, and Technical Services (25% of postings) alongside Healthcare and Social Assistance (18%) indicates a strong demand for specialized knowledge workers and healthcare professionals. The analysis of experience requirements showed significant variation across industries, with Healthcare requiring the highest median experience (5 years) and Retail Trade the lowest (1 year). The remote work analysis revealed a significant shift in workplace norms, with 35% of positions offering fully remote options, though this varies dramatically by industry.

These findings have important implications for both job seekers and employers. Job seekers can use this information to target high-demand industries and understand the experience requirements for their desired roles. Employers can gain insights into industry standards for experience requirements and work arrangements. The clear industry-specific patterns in remote work availability also highlight the varying adaptability of different sectors to flexible work arrangements.

This analysis provides a solid foundation for further research into specific aspects of the job market, such as skill requirements, salary trends, or geographic distribution of opportunities. 